# Benchmark comparison between PyRuSH vs PyRuSH[jre]

This notebook compares the PyRuSH vs PyRuSH[jre]. The former is the original implementation in python. The latter is implemented by calling RuSH jar through py4j interface. The results demonstrate signficantly difference in speed. So PyRuSH[jre] is recommened for operational use.

In [1]:
%cd ..

/Users/u0876964/Projects/DataspellProjects/PyRuSH


In [3]:
%%capture
!pip install MechanicalSoup

In [2]:
from pathlib import Path
import re
import mechanicalsoup
import html
import urllib.parse


## Create a dataset

In [19]:

# Connect to duckduckgo
browser = mechanicalsoup.StatefulBrowser(user_agent='MechanicalSoup')
browser.open("https://www.mtsamples.com/site/pages/browse.asp?type=91-SOAP%20/%20Chart%20/%20Progress%20Notes")

<Response [200]>

In [20]:
links=[]
for link in browser.links():
    target = link.attrs['href']
    if target.startswith('/site/pages/sample.asp?Type=91-SOAP / Chart / Progress Notes&Sample='):
        links.append(link)
print(len(links))

498


In [21]:
# just get 50 documents
for i,link in enumerate(links[:50]):
    browser.follow_link(link)
    page=browser.get_current_page()
    # this might not work if mtsamples updates the tags
    div=page.find('div',class_='hilightBold')
    if div is None:
        continue
    content='\n'.join(div.findAll(text=True, recursive=False)).strip()
    Path('data/doc{}.txt'.format(i)).write_text(content)

In [3]:
docs=dict()
for f in Path('data').glob('*.txt'):
    docs[f.stem]=f.read_text()

## Processing use PyRuSH[jre]

In [7]:
from PyRuSH import RuSH
rush = RuSH('conf/rush_rules.tsv')

In [9]:
%%timeit -n 10
for n, doc in docs.items():
    rush.segToSentenceSpans(doc)

476 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Try PyRuSH

Now restart the notebook kernel, and run pure python PyRuSH

In [ ]:
from pathlib import Path
import re
import mechanicalsoup
import html
import urllib.parse

In [4]:
%cd ../data

/Users/u0876964/Projects/DataspellProjects/PyRuSH/data


In [ ]:
docs=dict()
for f in Path('./').glob('*.txt'):
    docs[f.stem]=f.read_text()

In [18]:
%%capture
!pip install -I -U PyRuSH

In [28]:
from PyRuSH.RuSH import RuSH

In [29]:
rush = RuSH('../conf/rush_rules.tsv')

In [14]:
%%timeit -n 10
for n, doc in docs.items():
    rush.segToSentenceSpans(doc)

3.33 s ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
